# Convert ATM ICESSN v2 format to GeoDataFrames and GeoDataPackages with fractional year

The NASA Airborne Topographic Mapper (ATM) ICESSNv2 data contains resampled and smoothed elevation measurements from ATM's airborne lidar instruments.  
All data are freely available from the National Snow and Ice Data Center (data set ID: ILATM2).  
Download data: https://nsidc.org/data/ilatm2/versions/2  
DOI: 10.5067/CPRXXK3F39RV

**Note**: fractional years are truncated to the day of the survey without HH:MM:SS information. UTC seconds past midnight are preserved in the converted GPKG files and can be used to create UTC-based timetags for each platelet.

In [1]:
# import necessary modules

import os
import glob
import datetime
import numpy as np
import pandas as pd
import geopandas as gpd
from   datetime import timezone

loop over all ICESSN v2 files in a given folder and convert them to GPKG files with fractional year added

In [2]:
for file in glob.glob("/home/jovyan/shared-public/off_track/Sierra/ATM/ILATM2_*.csv"):

    f_name_gis = file.replace("csv","gpkg") # file name for export
    # read ICESSN v2 ASCII csv files into dataframe 
    icessn_df = pd.read_csv(file, skiprows = 11) # use 11 for version 2
     
    if icessn_df.shape[1] == 11:
        icessn_df.columns = ['sec', 'lat', 'lon', 'ele', 'slope_ns','slope_ew','rms_fit','n_pts_used','n_pts_edited','x_track_m','track_id']
        # drop columns that are not needed for this project
        icessn_df = icessn_df.drop(columns=['slope_ns','slope_ew','rms_fit','n_pts_used','n_pts_edited','x_track_m'])
    
    # need to wrap ATM longitudes to ±180° for exporting geographic coordinates 
    # 0° to 360° is not supported for GeoDataFrames
    icessn_df["lon"] = np.mod(icessn_df["lon"] - 180.0, 360.0) - 180.0
    
    # extract survey date from ICESSN file name
    f_parts_inp = os.path.split(file)
    f_name_date_str = f_parts_inp[1][7:15]
    year  = int(f_name_date_str[0:4])
    month = int(f_name_date_str[4:6])
    day   = int(f_name_date_str[6:8])
    
    # create datetime object of survey date
    survey_date = datetime.datetime(year,month,day).replace(tzinfo=timezone.utc)
    
    # calculate fractional year
    start = datetime.date(survey_date.year, 1, 1).toordinal()
    year_length = datetime.date(survey_date.year+1, 1, 1).toordinal() - start
    year_fractional = survey_date.year + float(survey_date.toordinal() - start) / year_length
    
    # populate lat lon geometry in GeoDataFrame
    atm_gdf = gpd.GeoDataFrame(icessn_df, geometry=gpd.points_from_xy(icessn_df["lon"], icessn_df["lat"]))
    
    # add fractional year to GeoDataFrame
    atm_gdf.insert(atm_gdf.shape[1], "year_fractional", year_fractional)
    
    # remove redundant lon lat columns
    atm_gdf = atm_gdf.drop(columns=["lon","lat"])
    
    # set up the coordinate system
    # EPSG:4326 WGS84 - World Geodetic System 1984, used in DGPS solutions for ATM geolocation
    atm_gdf = atm_gdf.set_crs("EPSG:4326")  #
    
    # export file as GeoDataPackage
    atm_gdf.to_file(f_name_gis, driver="GPKG")
    print(f_name_gis)

/home/jovyan/shared-public/off_track/Sierra/ATM/ILATM2_20180427_141100_smooth_nadir3seg_50pt.gpkg
/home/jovyan/shared-public/off_track/Sierra/ATM/ILATM2_20170422_135121_smooth_nadir3seg_50pt.gpkg
/home/jovyan/shared-public/off_track/Sierra/ATM/ILATM2_20170422_134948_smooth_nadir3seg_50pt.gpkg
/home/jovyan/shared-public/off_track/Sierra/ATM/ILATM2_20180427_123100_smooth_nadir3seg_50pt.gpkg
/home/jovyan/shared-public/off_track/Sierra/ATM/ILATM2_20170505_171110_smooth_nadir3seg_50pt.gpkg
/home/jovyan/shared-public/off_track/Sierra/ATM/ILATM2_20190914_131000_smooth_nadir3seg_50pt.gpkg
/home/jovyan/shared-public/off_track/Sierra/ATM/ILATM2_20190914_130900_smooth_nadir3seg_50pt.gpkg
/home/jovyan/shared-public/off_track/Sierra/ATM/ILATM2_20190914_132700_smooth_nadir3seg_50pt.gpkg
/home/jovyan/shared-public/off_track/Sierra/ATM/ILATM2_20170422_134643_smooth_nadir3seg_50pt.gpkg
/home/jovyan/shared-public/off_track/Sierra/ATM/ILATM2_20170505_170524_smooth_nadir3seg_50pt.gpkg
/home/jovyan/shared-